# 软管流固耦合模拟（浸入边界方法）

AERO-Suite里的**AERO-F**是流体仿真模拟器，这里是详细的[**AERO-F**教程](https://bitbucket.org/frg/aero-f/downloads/AERO-F.pdf)。
**AERO-S**是固体仿真模拟器，这里是详细的[**AERO-S**教程](https://bitbucket.org/frg/aero-s/downloads/AERO-S.pdf)。

本案例是使用浸入边界方法（EBM）对软管的流固耦合的仿真模拟。本案例将着重介绍AERO-Suite里的自适应网格方法。本次仿真的文件都在 `Beam` 文件夹中，
在这个文件中一般有3个子文件夹：`sources`, `simulations`和`data`。
- `sources`文件夹一般存放网格相关的源文件
- `simulations`文件夹一般存放仿真模拟的输入文件
- `data`文件夹一般存放仿真模拟过程中生成的一些文件。

以下是本次仿真模拟的详细步骤。
在开始本次仿真模拟前，我们需要安装相关的软件，请参加安装教程,比如[如何在北京大学未名一号上安装AERO-Suite](../../../Install/Install_PKU.ipynb)。
如果我们之前已经安装好相关软件，我们可以读取 [.bashrc_frg](../../../Install/bashrc_frg) 刷新当前shell环境，载入各个软件的命令
```shell
source ~/.bashrc_frg 
```

# 网格文件

由于我们使用的自适应网格算法，最新节点二分法(newest node bisection)，对初始网格的编号有一定的要求。 我们在 `sources` 文件夹中，用**python**生成长方体的计算区域，以及均匀的四面体网格

```shell
python generateMesh.py
```
我们能在 [generateMesh.py](../sources/generateMesh.py) 中的 `uniform3D()` 函数里，指定 `x`, `y`, `z` 数组，并指定`z`底部，`z`顶部，`x`底部，`x`顶部，`y`底部，`y`顶部的边界名称，生成结构网格 `fluid.top`。

固体网格 [StructureFile.include](./StructureFile.include)是一维的梁单元，以及我们固定了软管顶部的顶点的所有自由度。我们在后处理画图时使用的固体网格是 [Structure.top](../sources/Structure.top)。值得注意的是。仿真模拟中需要的固体信息都在固体的输入文件 [StructureFile.FSI](./StructureFile.FSI)，用**AERO-S**可以处理固体输入文件 [StructureFile.FSI](./StructureFile.FSI)生成相应的固体网格文件 `Structure.top`：

```shell
aeros -t StructureFile.FSI 
```
这里我们加了命令参数 `-t` ，在 [StructureFile.FSI](./StructureFile.FSI) 里仿真的名字为 Structure， 因而生成的文件是 `Structure.top`。

对于使用浸入边界方法的仿真模拟，我们需要输入流体求解器表示固体形状的浸入表面 [embeddedSurface.top](../sources/embeddedSurface.top)，这个浸入表面可以和固体表面网格相同，也能与之不同。本案例中，浸入表面表示了细长软管的形状(六边形的横截面)，而固体网格是用梁单元对软管的简化模型，因而他们并不相同。我们可以用
```shell
python generateBeam.py 
```
生成不同长度、形状的固体网格文件`StructureFile.include`和浸入表面文件`StructureFile.include` 长方体的计算区域

我们可以用**xp2exo**把 `.top` 网格文件转化为 `.exo` 文件进行可视化，这里我们展示了流体网格、浸入表面、和固体网格:
<img src="../../Figs/Beam_EBM.png" width="800" />


# 网格前处理

我们首先使用**partnmesh**软件，能把流体网格划分64个区域，使用64个核进行并行运算。这可以在`Beam`文件里用以下命令行完成

```shell
partnmesh sources/fluid.top 64
```

这会在`sources`文件夹中，生成有分区标注的`fluid.top.dec.64`文件。

如前面所言，对于使用浸入边界方法的仿真模拟，我们需要使用**matcher**去匹配浸入表面 [embeddedSurface.top](../sources/embeddedSurface.top) 里标注了是 *Moving* 的边界单元的格点（比如StickMovingSurface），和固体表面的网格 [matcher.top](../sources/matcher.top)。对于本案例，固体表面的网格就是固体的表格。这可以在`Beam`文件里用以下命令行完成

```shell
matcher  sources/embeddedSurface.top  sources/matcher.top -beam -output data/fluidmodel
```
这里 `-beam` 是表示固体结构里有两单元需要匹配。这会在 `data` 文件夹里生成流体模拟使用的匹配文件 `fluidmodel.match.fluid`、固体模拟使用的匹配文件 `fluidmodel.match.fem` 和供可视化的 `fluidmodel.match.top` 文件。其中匹配文件 `fluidmodel.match.fluid`里是匹配的浸入表面上的格点数目共602个，然后是格点在浸入表面中的编号，然后是它们到固体表面网格的距离向量； `fluidmodel.match.fem`里，首先是匹配的浸入表面上的格点数目共602个，然后是每个匹配了的浸入表面格点的匹配信息 $n,\xi,\eta,n_x,n_y,n_z$， 其中$n$是流体格点匹配的固体表面网格 [matcher.top](../sources/matcher.top)里三角形有限元对应的编号，$\xi,\eta$ 是对应的匹配点在梁单元有限元里的自然坐标，$n_x,n_y,n_z$ 是流体格点到固体表面匹配点的距离向量。用这些信息我们AERO-Suite能实现流体、固体求解器对于流体对固体作用力、固体表面位移的交互。

之后我们需要用**sower**去生成各个区域网格的详细信息，把这些信息文件储存在`data`文件夹中，这可以在`Beam`文件夹里用以下命令行完成

```shell
sower -fluid -mesh sources/fluid.top -match data/fluidmodel.match.fluid -dec sources/fluid.top.dec.64 -cpu 64 -cluster 64 -output data/fluidmodel
```

这会在 `data` 文件夹中生成以 `fluidmodel` 为前缀的文件，包括各区域的网格信息比如 `fluidmodel.msh1`，标注信息比如 `fluidmodel.dec1`，匹配信息比如 `fluidmodel.fluid.match1`， 各64个文件，以及CPU和分区的映射 `fluidmodel.64cpu` 和各个分区的连接信息 `fluidmodel.con`。

对于网格划分的前处理，我们可以把这些命令行写在 `Beam` 文件夹里的脚本文件里，比如[preprocess.sh](../preprocess.sh)。我们可以用以下命令行运行脚本

```shell
bash preprocess.sh
```


# 流固耦合计算
本次案例的流固耦合仿真计算总共包括以下4个仿真计算
- 网格预加密：对指定的求解区域进行网格预加密
- 流体模拟：假设固体是不动的，对流体做模拟仿真，生成流固耦合模拟仿真中流体的初始状态
- 流固耦合模拟
- 流固耦合模拟重启

## 网格预加密

我们首先对流体网格`fluid.top`进行预加密，这里我们在流体仿真的输入文件 [FluidFile.APriori](./FluidFile.APriori)中加入了自适应模块，我们在一个盒子（`Box`）进行预加密

```shell
under AdaptiveMeshRefinement {
    under SelectionCriterion[0] {
      Type = APriori;
      under RefinementDomain {
        Type = Box;
        under Box {
            X0 = -0.5;
            Y0 = -0.5;
            Z0 = -0.5;
            X1 = 3.5;
            Y1 = 0.5;
            Z1 = 8;
          }
    }
    MinimumEdgeLength = 0.09;
    NumLevels = 10;
    PreRefine = On;
  }

  Mode = NonDimensional; 
  RecomputeWallDistance = 2;  
  BisectionStrategy = Isotropic;
  MaximumRefinementIterations = 1000;
  Eps = 1e-3;
  MaximumCoarseningIterations = 1000;
  BalancingFrequency = 50;
  ItrFactor = 1000.0;
  UseUnsteadyStrategy = 1;
  Frequency = 100;
}
```

我们只进行一步模拟（`MaxIts = 1`）

```shell
under Time {
  Type = Implicit;
  MaxIts = 1;
  MaxTime = 0.2;
  ...
}
```

然后存下来加密过后的网格信息，用加密过后的网格作为初始网格开始随后的模拟，值得注意的是初始网格是不能被粗化的。
这里加密的网格会存放在重启文件夹里(`references.APriori`)

```shell
under Input {
  GeometryPrefix = "../data/fluidmodel";
  EmbeddedSurface = "../sources/embeddedSurface.top";
}

under Output {
  under Restart {
    Prefix   = "references.APriori/";
    Solution = "Solution.bin";
    RestartData = "Restart.data";
    Position = "Position.data";
    EmbeddedPosition = "EmbeddedPosition.data";
    Frequency = 0;
  }
  ...
}
```
它们包括各区域的网格信息比如 `fluidmodel.msh1` 和标注信息比如 `fluidmodel.dec1` 各64个文件，以及CPU和分区的映射 `fluidmodel.64cpu` 和各个分区的连接信息 `fluidmodel.con`。这里使用的前缀和输入文件（`Input`）的前缀一致。之后的模拟我们不再使用 `data` 文件里的网格信息文件。


对于网格预加密的仿真模拟，我们可以把这些命令行写在 `Beam` 文件夹里的脚本文件里，比如[run.APriori.sh](../run.APriori.sh)。我们可以用以下命令行运行脚本
```shell
bash run.APriori.sh
```
更多时候我们在集群上提交作业，我们需要用 [sbatch命令](https://hpc.pku.edu.cn/_book/guide/slurm/sbatch.html) 将作业提交到计算节点上执行，在`simulations`文件夹里，我们可以编写作业脚本文件 [Sbatch.APriori.sh](./Sbatch.APriori.sh) 申请64个计算节点，进行计算。可以用以下命令行提交作业
```shell
sbatch Sbatch.APriori.sh
```


## 流体模拟 

对于所有流固耦合问题，我们首先假设固体是不动的，对流体做模拟仿真，然后用流体仿真的结果作为流固耦合模拟仿真流体的初始状态。对于本次模拟，这个流体仿真的输入文件是 [FluidFile.Unsteady](./FluidFile.Unsteady)。这个流体仿真会使用之前网格预加密模拟在`references.APriori`中生成的网格文件，并把重启文件以及相应的网格文件存放在重启文件夹 `references.Unsteady` 里。生成的为重启的网格文件包括：各区域的网格信息比如 `fluidmodel.msh1` 和标注信息比如 `fluidmodel.dec1` 各64个文件，以及CPU和分区的映射 `fluidmodel.64cpu` 和各个分区的连接信息 `fluidmodel.con`。这里使用的前缀和输入文件（`Input`）的前缀一致。


```shell
under Input {
  GeometryPrefix = "references.APriori/fluidmodel";
  EmbeddedSurface = "../sources/embeddedSurface.top";
}                                                                                            
under Output {
  under Postpro {
    PostprocessSides = 1;
    Prefix   = "results.Unsteady/";
    Mach = "Mach.bin";
    EmbeddedSurfaceDisplacement = "../postpro.Unsteady/EmbeddedSurfaceDisplacement.xpost";
    Frequency = 1000;
  }
  under Restart {
    Prefix = "references.Unsteady/"; 
    Solution = "Solution.bin";
    RestartData = "Restart.data";
    Position = "Position.data";
    EmbeddedPosition = "EmbeddedPosition.data";
    Frequency = 0;
  }
}
```

相比之前的案例，中加入了自适应模块，我们在流固耦合边界使用到流固耦合边界的距离（`WallProximity`）进行加密，
我们在流体内部，使用海森矩阵估计出来的后验误差（`Hessian`）进行加密， 我们也可以选定加密区域，比如球形区域（`Sphere`），一般我问会避免在边界处加密网格。

```shell
under AdaptiveMeshRefinement {
 under SelectionCriterion[0] {
    Type = WallProximity;
    under WallProximityCriterion {
      OuterZone_d = 1.0;
      OuterZone_H = 0.4;
      InnerZone_D = 2.0e-2;
      InnerZone_H = 4.0e-3;
      DeltaPlus = 1.0;
      EdgeLengthMinMaxRatio = 0.4;
    }
    MinimumEdgeLength = 4e-3;
    NumLevels = 10;
    PreRefine = On;
  }
  under SelectionCriterion[1] {
      Type = Hessian;
      under HessianCriterion {
      SensorType = Velocity;
      Threshold = 1.0e-1;
       HessianConstruction = LeastSquares;
       CoarseningThreshold = 2.0e-2;
    }
    PreRefine = Off;
    MinimumEdgeLength = 1.5e-2;
    NumLevels = 1;
  }
 under RefinementDomain {
      Type = Sphere;
      under Sphere {
      	    Center_x = 0.0;
            Center_y = 0.0;
            Center_z = 30.00;
            Radius = 45;
      }
  }
  Mode = NonDimensional; 
  RecomputeWallDistance = 2;  
  BisectionStrategy = Isotropic;
  MaximumRefinementIterations = 1000;
  Eps = 1e-3;
  MaximumCoarseningIterations = 1000;
  BalancingFrequency = 50;
  ItrFactor = 1000.0;
  UseUnsteadyStrategy = 1;
  Frequency = 50;
}
```

对于本次流体模拟，我们可以把这些命令行写在 `Beam` 文件夹里的脚本文件里，比如[run.Unsteady.sh](../run.Unsteady.sh)。我们可以用以下命令行运行脚本
```shell
bash run.Unsteady.sh
```
更多时候我们在集群上提交作业，我们需要用 [sbatch命令](https://hpc.pku.edu.cn/_book/guide/slurm/sbatch.html) 将作业提交到计算节点上执行，在`simulations`文件夹里，我们可以编写作业脚本文件 [Sbatch.Unsteady.sh](./Sbatch.Unsteady.sh) 申请64个计算节点，进行计算。可以用以下命令行提交作业
```shell
sbatch Sbatch.Unsteady.sh
```

## 流固耦合模拟

对于软管的流固耦合模拟仿真，我们的流体部分仿真输入文件是 [FluidFile.FSI](./FluidFile.FSI)，这个流体部分的仿真会使用之前流体模拟在 `references.Unsteady` 中生成的网格文件，以及流体的解 `Solution` 作为初始状态，值得注意的是，和以往的流固耦合模拟不同的是，我们这里也是用了重启的数据文件 `RestartData`， 这里我们需要编辑之前流体模拟生成的用于重启的数据文件`Restart.data`，只保留自适应网格前最大格点数 `MaxNodePreAMR`，如下
```shell
under RestartParameters {
  MaxNodePreAMR = 67686;
}
```
这个格点数是在预加密后的网格格点数，这里表示我们不能去粗化预加密后的网格。我们的输入（`Input`）和输出（`Output`）模块如下，对于生成的用于重启的文件将会存放在 `references.FSI` 文件夹里，它们包括流体的解，浸入边界的位置等，以及相应的网格文件，这些网格文件使用的前缀和输入文件（`Input`）里的网格文件前缀一致。

```shell
under Input {
  GeometryPrefix = "references.Unsteady/fluidmodel";

  Matcher = "../data/fluidmodel.match.fluid";
  EmbeddedSurface = "../sources/embeddedSurface.top";
 
  Solution = "references.Unsteady/Solution.bin";
  RestartData = "references.Unsteady/Restart.data";

}

under Output {
  under Postpro {
    PostprocessSides = 1;
    Prefix   = "results.FSI/";
    Mach  = "Mach.bin";
    EmbeddedSurfaceDisplacement = "../postpro.FSI/EmbeddedSurfaceDisplacement.xpost";
    LiftandDrag  = "../postpro.FSI/LiftandDrag.out";
    Frequency = 1000;
  }
  under Restart {
    Prefix = "references.FSI/"; 
    Solution = "Solution.bin";
    RestartData = "Restart.data";
    Position = "Position.data";
    EmbeddedPosition = "EmbeddedPosition.data";
    Frequency = 1000;
  }
}
```
这里使用的自适应网格模块于之前流体模拟的自适应网格模块一致。

对于我们的固体部分的仿真输入文件是 [Structure.FSI](./Structure.FSI)，
本次仿真是流固耦合的模拟（AERO），我们选择耦合方式，`A6 0.5 0.125` ， 即流体求解器采用隐式格式、固体求解器采用隐式中点法则的二阶精度的交错求解算法。我们也需要提供流固耦合边界匹配的信息（MATCHER）。

```shell
AERO
A6 0.5 0.125
MATCHER "../data/fluidmodel.match.fem"
```

本次流固耦合模拟仿真，我们模拟到 `1.0e-2` 单位时间，之后从这里开始重启模拟。
我们将用 `RESTART` 命令指定重启的相关文件，我们把用于固体求解器重启的数据信息存在 `references.FSI` 文件夹里名为`RESTART_FEM` 的文件，每1000步存一次。

```shell
RESTART
"references.FSI/RESTART_FEM" 1000
```

对于本次流固耦合模拟，我们可以把这些命令行写在 `Beam` 文件夹里的脚本文件里，比如[run.FSI.sh](../run.FSI.sh)。我们可以用以下命令行运行脚本
```shell
bash run.FSI.sh
```
更多时候我们在集群上提交作业，我们需要用 [sbatch命令](https://hpc.pku.edu.cn/_book/guide/slurm/sbatch.html) 将作业提交到计算节点上执行，在`simulations`文件夹里，我们可以编写作业脚本文件 [Sbatch.FSI.sh](./Sbatch.FSI.sh) 申请64个计算节点，进行计算。可以用以下命令行提交作业
```shell
sbatch Sbatch.FSI.sh
```

## 流固耦合模拟重启

很多时候，由于比如模拟中断，我们需要重启模拟，这里我们将展示软管的流固耦合模拟的重启。
流体部分用于重启仿真的输入文件是 [FluidFile.FSI.Restart](./FluidFile.FSI.Restart)，这个流体部分的仿真会使用之前流体模拟在 `references.FSI` 中生成的网格文件，以及流体的解 `Solution` 作为初始状态，重启的数据文件 `RestartData`，和浸入边界的位置 `EmbeddedPosition` 

```shell
under Input {
  GeometryPrefix = "references.FSI/fluidmodel";
  Matcher = "../data/fluidmodel.match.fluid";
  EmbeddedSurface = "../sources/embeddedSurface.top";
  // Restart from FSI simulation
  Solution = "references.FSI/Solution.bin";
  RestartData = "references.FSI/Restart.data";
  EmbeddedPosition = "references.FSI/EmbeddedPosition.data";
}
```
我们的输出（`Output`）模块和之前的流固耦合 [FluidFile.FSI](./FluidFile.FSI)里的一致，输出文件会自动添加在原来的输出文件后面，用于重启的文件将会覆盖之前的用于重启的文件。

```shell                                                                                                
under Output {
  under Postpro {
    PostprocessSides = 1;
    Prefix   = "results.FSI/";
    Mach  = "Mach.bin";
    EmbeddedSurfaceDisplacement = "../postpro.FSI/EmbeddedSurfaceDisplacement.xpost";
    LiftandDrag  = "../postpro.FSI/LiftandDrag.out";
    Frequency = 1000;
  }
  under Restart {
    Prefix = "references.FSI/"; 
    Solution = "Solution.bin";
    RestartData = "Restart.data";
    Position = "Position.data";
    EmbeddedPosition = "EmbeddedPosition.data";
    Frequency = 1000;
  }
}
```

固体部分用于重启仿真的输入文件是 [Structure.FSI.Restart](./Structure.FSI.Restart)。本次重启的流固耦合模拟仿真，我们将模拟到 `2.0e-2` 单位时间。我们将用 `RESTART` 命令指定重启的相关文件，第一行输入表示，我们把用于固体求解器重启的数据信息存在 `references.FSI` 文件夹里名为`RESTART_FEM` 的文件，每1000步存一次。第二行输入表示，本次模拟需要读入重启文件 `references.FSI/RESTART_FEM`，对于新的输出 `OUTPUT`，我们将加上后缀 `.2` 避免覆盖了以前的输出。

```shell
RESTART
"references.FSI/RESTART_FEM" 1000
"references.FSI/RESTART_FEM" ".2"
OUTPUT
GDISPLAC "results.FSI/structure.GDISPLAC.xpost" 1000
```


对于本次流固耦合模拟，我们可以把这些命令行写在 `Beam` 文件夹里的脚本文件里，比如[run.FSI.Restart.sh](../run.FSI.Restart.sh)。我们可以用以下命令行运行脚本

```shell
bash run.FSI.Restart.sh
```
更多时候我们在集群上提交作业，我们需要用 [sbatch命令](https://hpc.pku.edu.cn/_book/guide/slurm/sbatch.html) 将作业提交到计算节点上执行，在`simulations`文件夹里，我们可以编写作业脚本文件 [Sbatch.FSI.Restart.sh](./Sbatch.FSI.Restart.sh) 申请64个计算节点，进行计算。可以用以下命令行提交作业

```shell
sbatch Sbatch.FSI.Restart.sh
```

## 结果后处理

我们需要首先用**sower**把**AERO-F**并行输出的结果拼接起来。我们可以在`simulations`中，用以下命令行拼接关于压强和马赫数的结果。

```shell
sower -fluid -merge -con ../data/fluidmodel.con -mesh ../data/fluidmodel.msh -result results.FSI/Pressure.bin -output postpro.FSI/Pressure

sower -fluid -merge -con ../data/fluidmodel.con -mesh ../data/fluidmodel.msh -result results.FSI/Mach.bin -output postpro.FSI/Mach
```

对于流体输出，拼接起来的文件或是浸入边界上量的文件是在 `postpro.FSI` 文件夹里面后缀为 `.xpost` 的文件，为了使用**paraview**做后处理，我们需要用**xp2exo**把后缀为 `.xpost` 的文件转化为后缀为 `.exo` 的文件；对于固体输出，我们也需要用**xp2exo**把结果转化为后缀为 `.exo` 的文件。我们可以在 `simulations` 中，用以下命令行进行转换

```shell
xp2exo ../sources/fluid.top     fluid_solution.FSI.exo     postpro.FSI/Pressure.xpost postpro.FSI/Mach.xpost 

xp2exo ../sources/embeddedSurface.top  embeddedSurface.FSI.exo  postpro.FSI/EmbeddedSurfaceDisplacement.xpost postpro.FSI/EmbeddedSurfacePressureCoefficient.xpost

xp2exo ../sources/Structure.top  structure_solution.FSI.exo  postpro.FSI/structure.GDISPLAC.xpost 
```


我们可以用[**paraview**](https://www.paraview.org/tutorials/)读入 `fluid_solution.FSI.exo` 、 `embeddedSurface.FSI.exo ` 和 `structure_solution.FSI.exo` 进行可视化。
这里我们用**paraview**展示了马赫场和AGARD机翼的形变以及上面的压力系数
<img src="../../../Figs/AGARD_EBM_FSI.png" width="300" />


## bash脚本运行

对于网格划分的前处理，我们可以把这些命令行写在 `AGARD_EBM` 文件夹里的脚本文件里，比如[preprocess.sh](../preprocess.sh)。我们可以用以下命令行运行脚本

```shell
bash preprocess.sh
```


对于运行**AERO-F**的定常流模拟以及结果的后处理，我们可以把这些命令行写在 `simulations` 文件夹里的脚本文件里。我们可以用以下命令运行脚本

```shell
bash run.Steady.sh
bash postprocess.Steady.sh
```

对于运行**AERO-F**和**AERO-S**的流固耦合模拟以及结果的后处理，我们可以把这些命令行写在 `simulations` 文件夹里的脚本文件里。我们可以用以下命令运行脚本

```shell
bash run.FSI.sh
bash postprocess.FSI.sh
```

为了在集群上提交作业，我们需要用 [sbatch命令](https://hpc.pku.edu.cn/_book/guide/slurm/sbatch.html) 将作业提交到计算节点上执行，在`simulations`文件夹里，我们可以编写作业脚本文件 [Sbatch.FSI.sh](./Sbatch.FSI.sh) 申请计算节点，进行计算。可以用以下命令行提交作业

```shell
sbatch Sbatch.FSI.sh
```



